In [ ]:
# @title Установка
import os
from google.colab import drive
#@markdown * Подключить гугл диск
google_drive = False # @param {"type":"boolean"}
# @markdown * Язык
language = "ru" # @param ["ru","en"]
repo_url = "https://github.com/noblebarkrr/mvsepless"
python_path = "python"
home_dir = os.path.join(os.sep, "content")
mvsepless_dir = os.path.join(home_dir, "mvsepless-gamma")
os.environ["MVSEPLESS_PYTHON_PATH"] = python_path
if google_drive == True:
    drive.mount('/content/drive')
    data_dir = "/content/drive/MyDrive/mvsepless-data"
    os.makedirs(data_dir, exist_ok=True)
    os.environ["MVSEPLESS_HISTORY_PATH"] = os.path.join(data_dir, "history.json")
    os.environ["MVSEPLESS_OUTPUT_DIR"] = os.path.join(data_dir, "output")
    os.environ["MVSEPLESS_DOWNLOAD_DIR"] = os.path.join(data_dir, "downloaded")
%cd $home_dir
!git clone $repo_url -b gamma $mvsepless_dir
%cd $mvsepless_dir
!pip install uv
!uv pip install --no-cache-dir -qq -r requirements.txt
%cd $mvsepless_dir
from multi_inference import MVSEPLESS
mv = MVSEPLESS(lang=language)


# Web-UI

In [ ]:
import os
from pyngrok import ngrok
import random
import string
import re
import urllib
import time
import ipywidgets as widgets
from IPython.display import display, Javascript
import threading


#@title Запуск основного интерфейса
#@markdown * Тема
theme = "gamma" # @param ["gamma","beta","alpha","mvsep","vbachgen"]
#@markdown * Поддержка плагинов
add_plugins = True # @param {type:"boolean"}
#@markdown * Активировать плагин для MVSEP API
mvsep_api = True # @param {type:"boolean"}
# @markdown * Загрузить плагин для преобразования голоса <br> ***Vbach - форк PolGen-RVC 1.2.0***<br>(Загружается один раз)
vbach = False # @param {"type":"boolean"}
#@markdown ---
#@markdown ### Общий доступ
port = 7860
#@markdown * Запустить в ячейке гугл колаба
inline = False # @param {"type":"boolean"}
#@markdown * Способ поделится приложением
sharing_method = "not" # @param ["gradio","ngrok","localtunnel","not"]
#@markdown * Токен для ngrok *(где взять его - https://dashboard.ngrok.com/get-started/your-authtoken)*
ngrok_token = "" # @param {"type":"string"}

lt_sub_domain = "mvsepless"
def generate_subdomain(length=8):
    """Генерация случайного субдомена заданной длины"""
    chars = string.ascii_lowercase + string.digits
    return ''.join(random.choice(chars) for _ in range(length))

if sharing_method == "ngrok":
    try:
        ngrok.set_auth_token(ngrok_token)
        ngrok.kill()
        tunnel = ngrok.connect(port)
        print(f"Публичная ссылка: {tunnel.public_url}")
    except KeyboardInterrupt:
        ngrok.kill()

if sharing_method == "localtunnel":
    os.system("npm install -g localtunnel &>/dev/null")
    time.sleep(7)
    with open('url.txt', 'w') as file:
        file.write('')
    subdomain = f"{re.sub(r'[^a-zA-Z0-9]', '', lt_sub_domain)}-{generate_subdomain(25)}"


    def run_tunnel():
        while True:
            print("localtunnel включается...")
            os.system(
                f'lt --port {port} '
                f'{f"--subdomain {subdomain}" if lt_sub_domain != "" or lt_sub_domain.isspace() == False else ""} '
            )

    tunnel = threading.Thread(target=run_tunnel, daemon=True)
    tunnel.start()

    time.sleep(3)
    try:
        endpoint_ip = urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip()
        tunnel_url = f"https://{subdomain}.loca.lt"
        print(f"Публичная ссылка: {tunnel_url}")
        text_field = widgets.Text(
            value=endpoint_ip,
            description='Пароль:',
            disabled=True
        )
        text_field.add_class("copy-enabled")

        display(text_field)
        display(Javascript("""
        setTimeout(() => {
            const input = document.querySelector('.copy-enabled input');
            if (!input) return;
            const btn = document.createElement('button');
            btn.innerHTML = '📋';
            btn.style.cssText = `
                margin-left: 8px;
                border: none;
                background: none;
                cursor: pointer;
                font-size: 1.2em;
            `;
            input.parentNode.appendChild(btn);
            btn.addEventListener('click', () => {
                navigator.clipboard.writeText(input);
                btn.innerHTML = '✓';
                setTimeout(() => btn.innerHTML = '📋', 2000);
            });
        }, 300);
        """))

    except Exception as e:
        print(f"Ошибка при старте localtunnel: {e}")

os.environ["MVSEP_API_PLUGIN"] = str(mvsep_api)

if vbach == True:
    !wget -O {os.path.join(mvsepless_dir, "plugins", "vbach_for_mvsepless_gamma_py312.py")}  https://huggingface.co/noblebarkrr/mvsepless_plugins/resolve/main/vbach_for_mvsepless_gamma_py312.py?download=true

app = mv.gradio_app(port=port, share=True if sharing_method == "gradio" else False, debug=True, inline=inline, plugins=add_plugins)
app.launch()

# CLI

## Разделение

### Выбор модели по типу и имени

In [ ]:


#@markdown ### Входные данные
#@markdown * Путь к входной папке/файлу:
input_path = "" # @param {type:"string"}
#@markdown ---
#@markdown ### Выбор модели
#@markdown * Тип / Имя модели:
model_name = "bs_roformer / BS-Roformer_SW" # @param ['mel_band_roformer / Mel-Band-Roformer_Vocals_kimberley_jensen', 'mel_band_roformer / Mel-Band-Roformer_InstVoc_Duality_v1_unwa', 'mel_band_roformer / Mel-Band-Roformer_InstVoc_Duality_v2_unwa', 'mel_band_roformer / Mel-Band-Roformer_Kim_FT_v1_unwa', 'mel_band_roformer / Mel-Band-Roformer_Kim_FT_v2_unwa', 'mel_band_roformer / Mel-Band-Roformer_Kim_FT_v2_bleedless_unwa', 'mel_band_roformer / Mel-Band-Roformer_Kim_FT_v3_prev_unwa', 'mel_band_roformer / Mel-Band-Roformer_Big_Beta_v1_unwa', 'mel_band_roformer / Mel-Band-Roformer_Big_Beta_v2_unwa', 'mel_band_roformer / Mel-Band-Roformer_Big_Beta_v3_unwa', 'mel_band_roformer / Mel-Band-Roformer_Big_Beta_v4_unwa', 'mel_band_roformer / Mel-Band-Roformer_Big_Beta_v5e_unwa', 'mel_band_roformer / Mel-Band-Roformer_Big_Beta_v6_unwa', 'mel_band_roformer / Mel-Band-Roformer_Big_Beta_v6x_unwa', 'mel_band_roformer / Mel-Band-Roformer_Instrumental_v1_unwa', 'mel_band_roformer / Mel-Band-Roformer_Instrumental_v1_plus_unwa', 'mel_band_roformer / Mel-Band-Roformer_Instrumental_v1e_unwa', 'mel_band_roformer / Mel-Band-Roformer_Instrumental_v1e_plus_unwa', 'mel_band_roformer / Mel-Band-Roformer_Instrumental_v2_unwa', 'mel_band_roformer / Mel-Band-Roformer_Small_v1_unwa', 'mel_band_roformer / Mel-Band-Roformer_Bleed_Suppressor_v1_unwa_97chris', 'mel_band_roformer / Mel-Band-Roformer_Instrumental_becruliy', 'mel_band_roformer / Mel-Band-Roformer_Guitar_becruily', 'mel_band_roformer / Mel-Band-Roformer_Karaoke_becruily', 'mel_band_roformer / Mel-Band-Roformer_Vocals_becruily', 'mel_band_roformer / Mel-Band-Roformer_SYHFT_v1_syh99999', 'mel_band_roformer / Mel-Band-Roformer_SYHFT_v2_syh99999', 'mel_band_roformer / Mel-Band-Roformer_SYHFT_v2.5_syh99999', 'mel_band_roformer / Mel-Band-Roformer_SYHFT_v3_syh99999', 'mel_band_roformer / Mel-Band-Roformer_BIG_SYHFT_v1_Fast_syh99999', 'mel_band_roformer / Mel-Band-Roformer_Merged_SYHFT_Beta_v1_syh99999', 'mel_band_roformer / Mel-Band-Roformer_SYHFT_B1_model1_syh99999', 'mel_band_roformer / Mel-Band-Roformer_SYHFT_B1_model2_syh99999', 'mel_band_roformer / Mel-Band-Roformer_SYHFT_B1_model3_syh99999', 'mel_band_roformer / Mel-Band-Roformer_4_stems_FT_Large_v1_syh99999', 'mel_band_roformer / Mel-Band-Roformer_4_stems_FT_Large_v2_syh99999', 'mel_band_roformer / Mel-Band-Roformer_Instrumental_1652_essid', 'mel_band_roformer / Mel-Band-Roformer_Instrumental_1681_essid', 'mel_band_roformer / Mel-Band-Roformer_Instrumental_Fv1_gabox', 'mel_band_roformer / Mel-Band-Roformer_Instrumental_Fv2_gabox', 'mel_band_roformer / Mel-Band-Roformer_Instrumental_Fv3_gabox', 'mel_band_roformer / Mel-Band-Roformer_Instrumental_Fv4N_gabox', 'mel_band_roformer / Mel-Band-Roformer_Instrumental_Fv5_gabox', 'mel_band_roformer / Mel-Band-Roformer_Instrumental_Fv5N_gabox', 'mel_band_roformer / Mel-Band-Roformer_Instrumental_Fv6_gabox', 'mel_band_roformer / Mel-Band-Roformer_Instrumental_Fv6N_gabox', 'mel_band_roformer / Mel-Band-Roformer_Instrumental_Fv7_gabox', 'mel_band_roformer / Mel-Band-Roformer_Instrumental_Fv7N_gabox', 'mel_band_roformer / Mel-Band-Roformer_Instrumental_Fv7_plus_gabox', 'mel_band_roformer / Mel-Band-Roformer_Instrumental_Fv7z_gabox', 'mel_band_roformer / Mel-Band-Roformer_Instrumental_Fv8_gabox', 'mel_band_roformer / Mel-Band-Roformer_Instrumental_Fv8b_gabox', 'mel_band_roformer / Mel-Band-Roformer_Instrumental_Fv9_gabox', 'mel_band_roformer / Mel-Band-Roformer_Instrumental_Fv10_gabox', 'mel_band_roformer / Mel-Band-Roformer_Instrumental_FvX_gabox', 'mel_band_roformer / Mel-Band-Roformer_Instrumental_Bv1_gabox', 'mel_band_roformer / Mel-Band-Roformer_Instrumental_Bv2_gabox', 'mel_band_roformer / Mel-Band-Roformer_Instrumental_Bv3_gabox', 'mel_band_roformer / Mel-Band-Roformer_Vocals_Fv1_gabox', 'mel_band_roformer / Mel-Band-Roformer_Vocals_Fv2_gabox', 'mel_band_roformer / Mel-Band-Roformer_Vocals_Fv3_gabox', 'mel_band_roformer / Mel-Band-Roformer_Vocals_Fv4_gabox', 'mel_band_roformer / Mel-Band-Roformer_Vocals_Fv5_gabox', 'mel_band_roformer / Mel-Band-Roformer_Karaoke_25_02_2025_gabox', 'mel_band_roformer / Mel-Band-Roformer_Karaoke_28_02_2025_gabox', 'mel_band_roformer / Mel-Band-Roformer_Karaoke_v1_gabox', 'mel_band_roformer / Mel-Band-Roformer_Karaoke_v2_gabox', 'mel_band_roformer / Mel-Band-Roformer_Lead_Vocals_DeReverb_gabox', 'mel_band_roformer / Mel-Band-Roformer_Denoise_DeBleed_gabox', 'mel_band_roformer / Mel-Band-Roformer_Karaoke_Fusion_gonzaluigi', 'mel_band_roformer / Mel-Band-Roformer_Karaoke_Fusion_Aggr_gonzaluigi', 'mel_band_roformer / Mel-Band-Roformer_BVE_gonzaluigi', 'mel_band_roformer / Mel-Band-Roformer_Karaoke_Fusion_v2_Aggr_gonzaluigi', 'mel_band_roformer / Mel-Band-Roformer_Karaoke_Fusion_Total_gonzaluigi', 'mel_band_roformer / Mel-Band-Roformer_DeReverb_anvuew', 'mel_band_roformer / Mel-Band-Roformer_DeReverb_Less_Aggr_anvuew', 'mel_band_roformer / Mel-Band-Roformer_DeReverb_Mono_anvuew', 'mel_band_roformer / Mel-Band-Roformer_Aspiration_sucial', 'mel_band_roformer / Mel-Band-Roformer_DeReverb-Echo_v1_sucial', 'mel_band_roformer / Mel-Band-Roformer_DeBigReverb_sucial', 'mel_band_roformer / Mel-Band-Roformer_DeSuperBigReverb_sucial', 'mel_band_roformer / Mel-Band-Roformer_DeReverb-Echo_Fused_sucial', 'mel_band_roformer / Mel-Band-Roformer_DeReverb-Echo_v2_sucial', 'mel_band_roformer / Mel-Band-Roformer_Karaoke_aufr33_viperx', 'mel_band_roformer / Mel-Band-Roformer_DeNoise_aufr33', 'mel_band_roformer / Mel-Band-Roformer_Denoise_Aggr_aufr33', 'mel_band_roformer / Mel-Band-Roformer_Crowd_aufr33_viperx', 'mel_band_roformer / Mel-Band-Roformer_Vocals_viperx', 'mel_band_roformer / Mel-Band-Roformer_Vocals_Fullness_aname', 'mel_band_roformer / Mel-Band-Roformer_Kim_FT_v1_aname', 'mel_band_roformer / Mel-Band-Roformer_Kim_FT_v2_aname', 'mel_band_roformer / Mel-Band-Roformer_Kim_FT_v2_Fullness_aname', 'mel_band_roformer / Mel-Band-Roformer_Kim_FT_v3_aname', 'mel_band_roformer / Mel-Band-Roformer_Small_aname', 'mel_band_roformer / Mel-Band-Roformer_Duality_v1_aname', 'mel_band_roformer / Mel-Band-Roformer_4_stems_Large_aname', 'mel_band_roformer / Mel-Band-Roformer_4_stems_v2_Large_aname', 'mel_band_roformer / Mel-Band-Roformer_4_stems_XL_aname', 'mel_band_roformer / Mel-Band-Roformer_Drums_yolkispaliks', 'mel_band_roformer / Mel-Band-Roformer_Instrumental_Metal_Preview_meskvlla33', 'bs_roformer / BS-Roformer_Drums_beatloo_labs', 'bs_roformer / BS-Roformer_Bass_beatloo_labs', 'bs_roformer / BS-Roformer_Vocals_1296_viperx', 'bs_roformer / BS-Roformer_Other_viperx', 'bs_roformer / BS-Roformer_Revive_v1_unwa', 'bs_roformer / BS-Roformer_Revive_v2_unwa', 'bs_roformer / BS-Roformer_Revive_v3e_unwa', 'bs_roformer / BS-Roformer_Resurrection_unwa', 'bs_roformer / BS-Roformer_Resurrection_Inst_unwa', 'bs_roformer / BS-Roformer_Inst_FNO_unwa', 'bs_roformer / BS-Roformer_Karaoke_becruily', 'bs_roformer / BS-Roformer_VocTest_gabox', 'bs_roformer / BS-Roformer_SW', 'bs_roformer / BS-Roformer_4_stems_zfturbo', 'bs_roformer / BS-Roformer_4_stems_FT_syh99999', 'bs_roformer / BS-Roformer_Chorus_Male-Female_146_sucial', 'bs_roformer / BS-Roformer_Chorus_Male-Female_267_sucial', 'bs_roformer / BS-Roformer_Male-Female_aufr33', 'bs_roformer / BS-Roformer_Deverb_256_8_anvuew', 'bs_roformer / BS-Roformer_Deverb_384_10_anvuew', 'bs_roformer / BS-Roformer_4_stems_aname', 'mdx23c / MDX23C_InstVoc_HQ_zfturbo', 'mdx23c / MDX23C_8kFFT_InstVoc_HQ_v1', 'mdx23c / MDX23C_8kFFT_InstVoc_HQ_v2', 'mdx23c / MDX23C_D1581', 'mdx23c / MDX23C_DrumSep_aufr33_jarredou', 'mdx23c / MDX23C_DeReverb_aufr33_jarredou', 'mdx23c / MDX23C_Mid_Side_wesleyr36', 'mdx23c / MDX23C_4_stems_zfturbo', 'mdx23c / MDX23C_Orchestra_verosment', 'scnet / SCNet_4_stems_zfturbo', 'scnet / SCNet_XL_IHF_4_stems_zfturbo', 'scnet / SCNet_XL_4_stems_starrytong', 'scnet / SCNet_XL_4_stems_zftrubo', 'scnet / SCNet_Large_Jazz_4_stems_jorisvaneyghen', 'scnet / SCNet_XL_Jazz_4_stems_jorisvaneyghen', 'vr / 1_HP-UVR', 'vr / 2_HP-UVR', 'vr / 3_HP-Vocal-UVR', 'vr / 4_HP-Vocal-UVR', 'vr / 5_HP-Karaoke-UVR', 'vr / 6_HP-Karaoke-UVR', 'vr / 7_HP2-UVR', 'vr / 8_HP2-UVR', 'vr / 9_HP2-UVR', 'vr / 10_SP-UVR-2B-32000-1', 'vr / 11_SP-UVR-2B-32000-2', 'vr / 12_SP-UVR-3B-44100', 'vr / 13_SP-UVR-4B-44100-1', 'vr / 14_SP-UVR-4B-44100-2', 'vr / 15_SP-UVR-MID-44100-1', 'vr / 16_SP-UVR-MID-44100-2', 'vr / 17_HP-Wind_Inst-UVR', 'vr / UVR-De-Echo-Aggressive', 'vr / UVR-De-Echo-Normal', 'vr / UVR-DeEcho-DeReverb', 'vr / UVR-DeNoise-Lite', 'vr / UVR-DeNoise', 'vr / UVR-BVE-4B_SN-44100-1', 'vr / UVR-BVE-v2-4B-SN-44100', 'vr / MGM-v5-KAROKEE-32000-BETA1', 'vr / MGM-v5-KAROKEE-32000-BETA2-AGR', 'vr / MGM_HIGHEND_v4', 'vr / MGM_LOWEND_A_v4', 'vr / MGM_LOWEND_B_v4', 'vr / MGM_MAIN_v4', 'vr / UVR-De-Reverb-aufr33-jarredou', 'vr / UVR-De-Breath-sucial-v1', 'vr / UVR-De-Breath-sucial-v2', 'vr / VR_Harmonic_Noise_Sep', 'mdx / UVR-MDX-NET-Inst_HQ_1', 'mdx / UVR-MDX-NET-Inst_HQ_2', 'mdx / UVR-MDX-NET-Inst_HQ_3', 'mdx / UVR-MDX-NET-Inst_HQ_4', 'mdx / UVR-MDX-NET-Inst_HQ_5', 'mdx / UVR_MDXNET_Main', 'mdx / UVR-MDX-NET-Inst_Main', 'mdx / UVR_MDXNET_1_9703', 'mdx / UVR_MDXNET_2_9682', 'mdx / UVR_MDXNET_3_9662', 'mdx / UVR-MDX-NET-Inst_1', 'mdx / UVR-MDX-NET-Inst_2', 'mdx / UVR-MDX-NET-Inst_3', 'mdx / UVR_MDXNET_KARA', 'mdx / UVR_MDXNET_KARA_2', 'mdx / UVR_MDXNET_9482', 'mdx / UVR-MDX-NET-Voc_FT', 'mdx / Kim_Vocal_1', 'mdx / Kim_Vocal_2', 'mdx / Kim_Inst', 'mdx / Reverb_HQ_By_FoxJoy', 'mdx / UVR-MDX-NET_Crowd_HQ_1', 'mdx / kuielab_a_vocals', 'mdx / kuielab_a_other', 'mdx / kuielab_a_bass', 'mdx / kuielab_a_drums', 'mdx / kuielab_b_vocals', 'mdx / kuielab_b_other', 'mdx / kuielab_b_bass', 'mdx / kuielab_b_drums', 'mdx / UVR-MDX-NET_Main_340', 'mdx / UVR-MDX-NET_Main_390', 'mdx / UVR-MDX-NET_Main_406', 'mdx / UVR-MDX-NET_Main_427', 'mdx / UVR-MDX-NET_Main_438', 'mdx / UVR-MDX-NET_Inst_82_beta', 'mdx / UVR-MDX-NET_Inst_90_beta', 'mdx / UVR-MDX-NET_Inst_187_beta', 'mdx / UVR-MDX-NET-Inst_full_292', 'htdemucs / HTDemucs4_MVSep_vocals', 'htdemucs / HTDemucs4', 'htdemucs / HTDemucs4_6s', 'htdemucs / Demucs3_mmi', 'htdemucs / HTDemucs4_FT_Bass', 'htdemucs / HTDemucs4_FT_Drums', 'htdemucs / HTDemucs4_FT_Vocals', 'htdemucs / HTDemucs4_FT_Other', 'htdemucs / HTDemucs4_Mid_Side_wesleyr36', 'bandit / Bandit_Plus', 'bandit_v2 / Bandit_v2_Multi']
# @markdown ---
# @markdown ### Настройки разделения
# @markdown  * Степень подавления для VR моделей
vr_arch_aggressive = 5 # @param {"type":"slider","min":1,"max":100,"step":1}
# @markdown * Извлечь  инструментал:
instrumental = True # @param {type:"boolean"}
#@markdown ---
#@markdown ### Выходные данные
#@markdown * Формат:
output_format = "mp3" # @param ["mp3", "wav", "flac", "ogg", "opus", "m4a", "aac", "aiff"]
# @markdown * Битрейт
bitrate = 320 # @param {"type":"slider","min":32,"max":320,"step":1}
# @markdown * Выбрать выходные стемы(через пробел, например ("vocal"  "instrumental")):
stems_to_extract = "" # @param {type:"string"}
#@markdown * Шаблон именования выходных файлов:
output_template = "NAME (STEM) MODEL" # @param {type:"string"}
#@markdown * Путь к выходной папке:
output_dir = "/content/output" # @param {type:"string"}
# @markdown * Показать список выходных файлов
show_list_outputs = True # @param {"type":"boolean"}

model = model_name.split(" / ")

%cd $mvsepless_dir

cmd = [
    f"{python_path}",
    "multi_inference.py separate",
    f"-i \"{input_path}\"",
    f"-o \"{output_dir}\"",
    f"-mt \"{model[0]}\"",
    f"-mn \"{model[1]}\"",
    f"-of \"{output_format}\"",
    f"-bitrate \"{bitrate}k\"",
    f"-vr_aggr {vr_arch_aggressive}",
    f"--template \"{output_template}\""
]

if instrumental:
    cmd.append("--instrumental")

if show_list_outputs:
    cmd.append("-l_out")

if stems_to_extract:
    cmd.append(f"--stems {stems_to_extract}")

!{" ".join(cmd)}

/content/mvsepless-gamma
Starting inference: bs_roformer/BS-Roformer_SW, bitrate=320k, method=cli, stems=[]
Model already downloaded
Разделение выполняется на ядрах CUDA. Для выполнения на процессоре установите force_cpu=True.
Используется устройство: cuda:0
/usr/local/lib/python3.12/dist-packages/rotary_embedding_torch/rotary_embedding_torch.py:35: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled = False)
/usr/local/lib/python3.12/dist-packages/rotary_embedding_torch/rotary_embedding_torch.py:268: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled = False)
Выбранный чекпоинт: /content/mvsepless-gamma/separator/models_cache/bs_roformer/BS-Roformer_SW.ckpt
Стемы: ['bass', 'drums', 'other', 'vocals', 'guitar', 'piano']
Потрачено времени на загрузку модели: 4.63 сек.
Выбранное аудио: /content/lexycat — gli

### Выбор модели по ID

In [ ]:


#@markdown ### Входные данные
#@markdown * Путь к входной папке/файлу:
input_path = "" # @param {type:"string"}
#@markdown ---
#@markdown ### Выбор модели
#@markdown * ID модели:
model_id = 217 # @param {"type":"integer"}
# @markdown ---
# @markdown ### Настройки разделения
# @markdown  * Степень подавления для VR моделей
vr_arch_aggressive = 5 # @param {"type":"slider","min":1,"max":100,"step":1}
# @markdown * Извлечь  инструментал:
instrumental = True # @param {type:"boolean"}
#@markdown ---
#@markdown ### Выходные данные
#@markdown * Формат:
output_format = "mp3" # @param ["mp3", "wav", "flac", "ogg", "opus", "m4a", "aac", "aiff"]
# @markdown * Битрейт
bitrate = 320 # @param {"type":"slider","min":32,"max":320,"step":1}
# @markdown * Выбрать выходные стемы(через пробел, например ("vocal"  "instrumental")):
stems_to_extract = "" # @param {type:"string"}
#@markdown * Шаблон именования выходных файлов:
output_template = "NAME (STEM) MODEL" # @param {type:"string"}
#@markdown * Путь к выходной папке:
output_dir = "/content/output" # @param {type:"string"}
# @markdown * Показать список выходных файлов
show_list_outputs = True # @param {"type":"boolean"}



%cd $mvsepless_dir

cmd = [
    f"{python_path}",
    "multi_inference.py id_separate",
    f"-i \"{input_path}\"",
    f"-o \"{output_dir}\"",
    f"-m_id {model_id}",
    f"-of \"{output_format}\"",
    f"-bitrate \"{bitrate}k\"",
    f"-vr_aggr {vr_arch_aggressive}",
    f"--template \"{output_template}\""
]

if instrumental:
    cmd.append("--instrumental")

if show_list_outputs:
    cmd.append("-l_out")

if stems_to_extract:
    cmd.append(f"--stems {stems_to_extract}")

!{" ".join(cmd)}

/content/mvsepless-gamma
Starting inference: bs_roformer/BS-Roformer_SW, bitrate=320k, method=cli, stems=[]
Model already downloaded
Разделение выполняется на ядрах CUDA. Для выполнения на процессоре установите force_cpu=True.
Используется устройство: cuda:0
/usr/local/lib/python3.12/dist-packages/rotary_embedding_torch/rotary_embedding_torch.py:35: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled = False)
/usr/local/lib/python3.12/dist-packages/rotary_embedding_torch/rotary_embedding_torch.py:268: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled = False)
Выбранный чекпоинт: /content/mvsepless-gamma/separator/models_cache/bs_roformer/BS-Roformer_SW.ckpt
Стемы: ['bass', 'drums', 'other', 'vocals', 'guitar', 'piano']
Потрачено времени на загрузку модели: 4.63 сек.
Выбранное аудио: /content/lexycat — gli

## Информация о моделях

In [4]:
#@title Показать список моделей
#@markdown * Фильтр по стему
filter = "" # @param {"type":"string"}
%cd $mvsepless_dir
if filter or filter != "":
    filter = f"-l_filter '{filter}'"
else:
    filter = ""


cmd = f"{python_path} multi_inference.py list {filter}"
!$cmd

/content/mvsepless-gamma
+-------------------+------+---------------------------------------------------------+-------------------------------------------+----------------------+
| Тип модели        |   ID | Имя модели                                              | Стемы                                     | Целевой инструмент   |
+===================+======+=========================================================+===========================================+======================+
| mel_band_roformer | 1000 | Mel-Band-Roformer_Vocals_kimberley_jensen               | vocals, other                             | vocals               |
+-------------------+------+---------------------------------------------------------+-------------------------------------------+----------------------+
| mel_band_roformer | 1010 | Mel-Band-Roformer_InstVoc_Duality_v1_unwa               | Vocals, Instrumental                      |                      |
+-------------------+------+-----------------------

## Загрузка аудио

In [ ]:
#@title Через yt-dlp
%cd $mvsepless_dir

url = "" # @param {"type":"string","placeholder":"Ссылка на аудио (youtube.com, soundcloud.com)"}
cookie = "" # @param {"type":"string","placeholder":"Путь к cookies-файлу"}

try:
    f = mv.downloader_audio.dw_yt_dlp(
        url = url,
        cookie = cookie if cookie != "" else None
    )
    print(f"Downloaded audio in: {f}")
except Exception as e:
    print(e)